# 딥러닝 실습 환경 소개

In [1]:
import tensorflow as tf

### 하이퍼 파라미터 설정

In [2]:
# Specify hyperparameters
EPOCHS = 5

### 네트워크 구조 정의  

In [4]:
# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

### 학습루프 정의

In [6]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

### 테스트 루프 정의

In [8]:
# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

### 데이터셋 가져오고, 정리

In [9]:
# Import and organize dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 네트워크 생성

In [10]:
# Create model
model = MyModel()

### 손실함수, 최적화 알고리즘 정의

In [11]:
# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### 알고리즘 평가지표 설정

In [12]:
# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### 알고리즘 학습, 테스트 루프

In [13]:
# Do training loop and test
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.3994556963443756, Accuracy: 89.56666564941406, Test Loss: 0.22647160291671753, Test Accuracy: 93.63999938964844
Epoch 2, Loss: 0.19680273532867432, Accuracy: 94.29000091552734, Test Loss: 0.16255196928977966, Test Accuracy: 95.2300033569336
Epoch 3, Loss: 0.1429569274187088, Accuracy: 95.85499572753906, Test Loss: 0.1288098394870758, Test Accuracy: 96.11000061035156
Epoch 4, Loss: 0.10956445336341858, Accuracy: 96.91000366210938, Test Loss: 0.11035218089818954, Test Accuracy: 96.77999877929688
Epoch 5, Loss: 0.0873815193772316, Accuracy: 97.51667022705078, Test Loss: 0.09816747903823853, Test Accuracy: 96.97000122070312
